In [ ]:
import numpy as np
import pandas as pd
import re
import string
import nltk
import os
import warnings
warning.filterwarning("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

from nltk import pos_tag
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import gensim 
from gensim.models import Word2Vec 

import keras
from keras.preprocessing import sequence
from keras.layers import Dropout, Dense, Flatten, Embedding,Activation
from keras.models import Sequential
from keras.layers.convolutional import Conv1D, MaxPooling1D 
from keras.layers import GlobalMaxPooling1D 
np.random.seed(20)

In [ ]:
os.chdir(r'C:\Users\oyo\Desktop')
df=pd.read_excel(r'training_29_6.xlsx',encoding='ISO-8859-1')
df.head()

In [ ]:
stemmer2 = SnowballStemmer('english')
## preprocessing text:
stop_words = set(stopwords.words('english'))
filter_SW = {"very","until","out","than",'ain','against','aren',"aren't","arent",'couldn',"couldn't","couldnt","didn","didn't","didnt","doesn","doesn't","doesnt","don","don't","dont","hadn","hadn't","hadnt","hasn","hasn't","haven","haven't","hasnt","isn","isn't","isnt","mightn't","mightnt","mightn","mustn","mustn't","mustnt","needn","needn't","neednt","no","not","nor","off","shan","shan't","shant","shouldn","shouldn't","shouldnt","wasn't","wasnt","wasn","weren","weren't","werent","won't","wont","won","wouldn","wouldn't","wouldnt"}
Nstop_words = stop_words - filter_SW
Nstop_words.update(["'","' "," '","’"," ’","’ "])
punct = ""
for i in string.punctuation:
    if(i!="'" ):
        punct = punct+i
#print(punct)

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
def preprocessing_text(text,lemma=True):
    text = text.lower()
    #replacing apostrophe with none
    text = text.replace("’","")
    text = text.replace("'","")
    text = text.replace('/','')
    #replacing special characters with a space
    text = re.sub('[^A-Za-z]+', " ", text)
    
    #replacing newline,tabs with none
    text = re.sub(r"[\n\t]*", "", text)
    
    #removing multiple sapces
    text = re.sub(" +"," ", text)
    
    #removing punctuations except apostrophe
    text = [word.strip(punct) for word in text.split(" ")]
    
    # remove words that contain numbers
    #text = [word for word in text if not any(c.isdigit() for c in word)]
    
    #removing stopwords
    text = [x for x in text if x not in Nstop_words]
    
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    
    # pos tag text
    pos_tags = pos_tag(text)
    
    # lemmatize text
    if lemma==True:
        text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
        
    else:
        ps = PorterStemmer()
        text =  [stemmer2.stem(word) for word in text]
        
    text = [t for t in text if len(t) > 1]
    text = " ".join(text)
    
    return(text)

In [ ]:
stem=[]
for i in range(len(df)):
    t=preprocessing_text(str(df['Review'][i]),lemma=False)
    stem.append(t)

In [ ]:
tokenizer = Tokenizer(num_words= 2000)
tokenizer.fit_on_texts(stem)
#sequences = tokenizer.texts_to_matrix(stem)
sequences = tokenizer.texts_to_sequences(stem)
seq = sequence.pad_sequences(sequences , maxlen = 60,padding = 'post', truncating = 'post' )

print("shape: ",np.shape(seq))
print(seq)

In [ ]:
reshape = keras.layers.Reshape
model = Sequential()
model.add(Embedding(input_dim = 2000, output_dim = 10 , input_length = 60))
model.add(Dropout(0.1))
model.add(Conv1D(64,kernel_size=3, strides = 1,activation='relu', padding = 'valid'))
model.add(MaxPooling1D(pool_size = 2, padding = 'same'))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
df1=pd.read_excel('test_18_6.xlsx')
text=df1['Review'][:]
'''
model1 = SentimentIntensityAnalyzer()
sentiment = pd.DataFrame(columns=['neg','neu','pos','compound'])
for i in range(len(text)):
    sentence = text[i]
    sentence = str(sentence)
    ss = model1.polarity_scores(sentence)
    sentiment =  sentiment.append(ss , ignore_index = True)
result = pd.concat([df1,sentiment], axis = 1)
result['sentiment']=result['compound'].apply(lambda x: 0  if x<0.5 else np.nan )
#result.head(10)
nan_rows = result[result['sentiment'].isnull()].index.values
result = result.drop(nan_rows)
#result.head()
result.to_excel('output_test1.xlsx',encoding='utf-8',index=False)
#result.head()'''

In [ ]:
'''df2 = pd.read_excel('output_test1.xlsx')
index = []
for i in range(len(df2['Review'])):
    text = df2['Review'][i]
    wordCount = len(text.split())
    if wordCount < 3 or wordCount >60:
        index.append(i)
df2 = df2.drop(index)
df2.to_excel('output_test2.xlsx',encoding = 'utf-8',index = False)
df2.head()'''

In [ ]:
'''
#testr = pd.read_excel('output_test2.xlsx')
#testing=testr
output=pd.DataFrame(columns=['AC/Heater','Check-in Experience', 'Comfort & Safety', 'Food Experience', 'Hotel Infrastructure','Hygiene & Cleanliness','Room Equipment & Amenities','Staff & Service','TV & WiFi','Washroom'])
result = pd.concat([df1, output], axis=1)
result.head()
'''

In [ ]:
'''
com=[]
for i in range(len(result)):
    t=preprocessing_text(result['Review'][i])
    com.append(t)
seqcs = tokenizer.texts_to_sequences(com)
seq1 = sequence.pad_sequences(seqcs , maxlen = 60)
'''

In [ ]:
#output=pd.DataFrame(columns=['AC/Heator_Predict','Check-in Experience_Predict', 'Comfort & Safety_Predict', 'Food Experience_Predict', 'Hotel Infrastructure_Predict','Hygiene & Cleanliness_Predict','Room equipment & Amenties_Predict','Staff & Service_Predict','TV & WiFi_Predict','Washroom_Predict'])
#final = pd.concat([result, output], axis=1)
#final.head()

In [ ]:
#add other classes
add_df=df[['AC/Heater', 'Check-in Experience',
       'Comfort & Safety', 'Food Experience', 'Hotel Infrastructure',
       'Hygiene & Cleanliness', 'Room Equipment & Amenities', 'Staff & Service',
       'TV & WiFi', 'Washroom']]
#add_dF=result[['AC/Heater', 'Check-in Experience',
#       'Comfort & Safety', 'Food Experience', 'Hotel Infrastructure',
#       'Hygiene & Cleanliness', 'Room Equipment & Amenities', 'Staff & Service',
#       'TV & WiFi', 'Washroom']]

dict_labels={'y1_train':'AC/Heater',
'y2_train':'Check-in Experience',
'y3_train':'Comfort & Safety',
'y4_train':'Food Experience',
'y5_train':'Hotel Infrastructure',
'y6_train':'Hygiene & Cleanliness',
'y7_train':'Room Equipment & Amenities',
'y8_train':'Staff & Service',
'y9_train':'TV & WiFi',
'y10_train':'Washroom',
}
'''
dict_labels2={'y1_train':'AC/Heater',
'y2_train':'Check-in Experience',
'y3_train':'Comfort & Safety',
'y4_train':'Food Experience',
'y5_train':'Hotel Infrastructure',
'y6_train':'Hygiene & Cleanliness',
'y7_train':'Room Equipment & Amenities',
'y8_train':'Staff & Service',
'y9_train':'TV & WiFi',
'y10_train':'Washroom',
}'''

for i in dict_labels:
    print("-------"+dict_labels[i]+"-------")
    X_train,X_test,Y_train,Y_test = train_test_split(seq,df[dict_labels[i]], test_size = 0.05)
    x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size=0.08)
    #x_test = seq1
    #y_test = result[dict_labels[i]]
    model.fit(x_train, y_train,batch_size = 32, validation_data=(x_val, y_val), class_weight = 'balanced', epochs=5)
    
    #y_score1= model.predict(x_test)
    #y_score = clf.predict(x_test)
    #print(y_score)
    #for j in range(len(y_score1)):
        #if y_score1[j] <= 0.5:
        #    result[dict_labels[i]][j] = 0
        #else:
        #    result[dict_labels[i]][j] = 1
    #result[dict_labels[i]] = y_score1
    y_sc = model.predict(X_test)
    y_score = np.zeros(len(y_sc))
    for i in range(len(y_sc)):
        if y_sc[i] < 0.51:
            y_score[i] = 0
        else:
            y_score[i] = 1
            
    #y_score = y_sc.apply(lambda x: 0  if x<0.5 else 1 )
    
    print("The accuracy score is: {}".format(str(accuracy_score(Y_test,y_score))))
    print("The roc_auc_score is: {}".format(str(roc_auc_score(Y_test, y_sc))))
    print(confusion_matrix(Y_test,y_score))  
    print(classification_report(Y_test,y_score))
    '''
     print("-------"+dict_labels[i]+"-------")
    print(accuracy_score(y_test,y_score))
    print(confusion_matrix(y_test,y_score))  
    print(classification_report(y_test,y_score))'''